In [1]:
import argparse
import os
import numpy as np
import pandas as pd
import math

from skimage import io
    
import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torch.autograd import Variable

from utils import draw_it
import torch.nn as nn
import torch.nn.functional as F
import torch
import ast

In [2]:
class QuickDrawDataset(Dataset):
    """Quick Draw dataset."""

    def __init__(self, csv_file, root_dir, to_image=None, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.data_frame = pd.read_csv(csv_file)
        self.data_frame['drawing'] = self.data_frame['drawing'].apply(ast.literal_eval)
        
        self.root_dir = root_dir
        self.to_image = to_image
        self.transform = transform
        
    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        image = np.array(self.data_frame.loc[idx].drawing)
        label = self.data_frame.loc[idx].word
        
        if self.transform:
            image = self.transform(image)
        elif self.to_image:
            image = draw_it(self.data_frame.loc[idx].drawing) 
    
        return image, label

In [ ]:
img_shape = (1, 32, 32)
cuda = True if torch.cuda.is_available() else False
class Generator(nn.Module):
    def __init__(self, n_classes):
        super(Generator, self).__init__()

        self.label_emb = nn.Embedding(n_classes, n_classes)

        def block(in_feat, out_feat, normalize=True):
            layers = [  nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(100+n_classes, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, noise, labels):
        # Concatenate label embedding and image to produce input
        gen_input = torch.cat((self.label_emb(labels), noise), -1)
        img = self.model(gen_input)
        img = img.view(img.size(0), *img_shape)
        return img

class Discriminator(nn.Module):
    def __init__(self, n_classes):
        super(Discriminator, self).__init__()

        self.label_embedding = nn.Embedding(n_classes, n_classes)

        self.model = nn.Sequential(
            nn.Linear(n_classes + int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1)
        )

    def forward(self, img, labels):
        # Concatenate label embedding and image to produce input
        d_in = torch.cat((img.view(img.size(0), -1), self.label_embedding(labels)), -1)
        validity = self.model(d_in)
        return validity

adversarial_loss = torch.nn.MSELoss()
auxiliary_loss = torch.nn.CrossEntropyLoss()

# Initialize generator and discriminator
generator = Generator(1)
discriminator = Discriminator(1)

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()
    auxiliary_loss.cuda()

# Configure data loader
data = QuickDrawDataset(csv_file='../train/line.csv', root_dir='train/', to_image=False, 
                        transform=transforms.ToTensor())
train_loader = DataLoader(dataset=data, batch_size=1, shuffle=False, num_workers=1)

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=.0002, betas=(0.5, .999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=.002, betas=(0.5, .999))

FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if cuda else torch.LongTensor

In [ ]:
num_epoch = 1
for epoch in range(num_epoch):
    for i, (batch_imgs, labels) in enumerate(train_loader):
        if i > 1: break
        batch_size = len(batch_imgs)

        # Adversarial ground truths
        valid = Variable(FloatTensor(batch_size, 1).fill_(1.0), requires_grad=False)
        fake = Variable(FloatTensor(batch_size, 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(batch_imgs.type(FloatTensor))
        #labels = Variable(labels.type(LongTensor))

In [3]:
data = QuickDrawDataset(csv_file='../train/line.csv', root_dir='train/', to_image=False, 
                        transform=transforms.ToTensor())

In [13]:
data[0]

(tensor([[[  0,   7]],
 
         [[126,   1]],
 
         [[190,   4]],
 
         [[255,   0]]]), 'line')

In [14]:
t = QuickDrawDataset(csv_file='../train/line.csv', root_dir='train/', to_image=False)

In [17]:
draw_it(t[0][0])

ValueError: source code string cannot contain null bytes

In [6]:
torch.tensor(t[0][0])

tensor([[[  0, 126, 190, 255],
         [  7,   1,   4,   0]]])

In [8]:
x,  y = t[0][0], t[0][1]

[[[  0 126 190 255]
  [  7   1   4   0]]] line


In [10]:
x.reshape((2, 4, 1))

array([[[  0],
        [126],
        [190],
        [255]],

       [[  7],
        [  1],
        [  4],
        [  0]]])

In [21]:
np.load('line.npy').shape

(143549, 784)